# More on Types, Methods, and Modules

Julia has a rich built-in type system, and most data types can be parameterized, such as
Array{Float64, 2} or Dict{Symbol, Float64}. Typing a variable (or more exactly the
value it is bound to) is optional.

However, indicating the type of some variables, although
they are not statically checked, can provide some of the advantages of static-type systems
as in C++, Java, or C#

A Julia program can run without any indication of types, which can
be useful in a prototyping stage, and it will still run fast.

However, some type indications
can increase the performance by allowing more specialized multiple dispatch.

Type
assertions also help the LLVM compiler to create more compact, better optimized code.

Moreover, typing function parameters makes the code easier to read and understand. The
robustness of the program is also enhanced by throwing exceptions, in cases where certain
type operations are not allowed. These failures will manifest themselves during testing, or
the code can provide an exception handling mechanism.

All functions in Julia are inherently generic or polymorphic, that is, they can operate on
different types of their arguments. The most appropriate method (an implementation of the
function where argument types are indicated) will be chosen at runtime to be executed,
depending on the type of arguments passed to the function

In this chapter, webroaden the previous discussions by covering the following topics:
- Type annotations
- The type hierarchy—subtypes and supertypes
- Concrete and abstract types
- User-defined and composite types
- Types and collections—inner constructors
- Type unions
- Parametric types and methods
- Standard modules and paths

Type-annotating a variable is done
with the :: operator, such as in the function definition function write(io::IO,
s::String)

To put it differently, io has to be an instance of type IO, and s an instance of type
String.

The :: operator is, in fact, an assertion that affirms that the value on the left is of
the type on the right. If this is not true, a typeassert error is thrown. Try this out in the
REPL:

In [3]:
"String"::Float64

LoadError: TypeError: in typeassert, expected Float64, got a value of type String

This is, in addition to the method specialization for multiple dispatch, an important reason
why type annotations are used in function signatures.

The operator :: can also be used in the sense of a type declaration, but only in local scope,
such as in functions, as follows:

In [4]:
function fun()
    a::Int64 = 23
end


fun (generic function with 1 method)

In [5]:
fun()

23

In [6]:
typeof(ans)

Int64

Every value assigned to n will be implicitly converted to the indicated type with the
convert function.

# Type conversions and promotions

The convert function can also be used explicitly in the code as convert(Int64, 7.0),
which returns 7.

In [9]:
convert(Int64, "7")

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mInt64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at /opt/julia-1.7.1/share/julia/base/number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at /opt/julia-1.7.1/share/julia/base/number.jl:7
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at /opt/julia-1.7.1/share/julia/base/twiceprecision.jl:262
[0m  ...

In [10]:
convert(String, 7)

LoadError: MethodError: [0mCannot `convert` an object of type [92mInt64[39m[0m to an object of type [91mString[39m
[0mClosest candidates are:
[0m  convert(::Type{String}, [91m::String[39m) at /opt/julia-1.7.1/share/julia/base/essentials.jl:223
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractString at /opt/julia-1.7.1/share/julia/base/strings/basic.jl:231
[0m  convert(::Type{T}, [91m::AbstractString[39m) where T<:AbstractString at /opt/julia-1.7.1/share/julia/base/strings/basic.jl:232
[0m  ...

In [11]:
convert(Int64, 3.141592)

LoadError: InexactError: Int64(3.141592)

In [12]:
convert(Int64, 3.0)

3

In general, convert(Type, x) will attempt to put the x value in an instance of Type. In
most cases, type(x) will also do the trick, as in Int64(7.0), returning 7.

In [13]:
Int64(3.0)

3

The conversion, however, doesn't always work:

- When precision is lost—Int64(7.01)returns an ERROR:
InexactError() error message

- When the target type is incompatible with the source value—convert(Int64,
"CV") returns an ERROR: MethodError: Cannot `convert` an object of
type String to an object of type Int64 error message

he types of the input
arguments are matched against the methods available for that function.

We can define our own conversions by providing new methods for the convert function.

In [22]:
903::Int64

903

In [31]:
11 % 10

1

In [32]:
div(11,10)

1

In [39]:
Int64('1')

49

In [40]:
Char(48)

'0': ASCII/Unicode U+0030 (category Nd: Number, decimal digit)

In [58]:
function custom_convert(type::DataType,number::Int64)
    if type != String throw("Sorry :(") end
    s = ""
    while true
        digit = number % 5
        s = string(s,Char(48+digit))
        number = div(number, 5)
        if number == 0
            return reverse(s)
        end
    end
end

            

custom_convert (generic function with 1 method)

In [59]:
custom_convert(String, 12)

"22"

Julia has a built-in system called automatic type promotion to promote arguments of
mathematical operators and assignments to a common type: in 4 + 3.14, the integer 4 is
promoted to a Float64 value, so that the addition can take place and results in
7.140000000000001.

In general, promotion refers to the conversion of values of different
types to one common type. This can be done with the promote function, which takes a
number of arguments, and returns a tuple of the same values, converting them to a
common type.

In [60]:
promote(1, 2.5, 3//4)

(1.0, 2.5, 0.75)

Thanks to the automatic type promotion system for numbers, Julia doesn't have to define,
for example, the + operator for any combinations of numeric types

In [67]:
# +(x::Number, y::Number) = +(promote(x, y)...)

It basically says: first, promote the arguments to a common type, and then perform the
addition. Number is a common supertype for all values of numeric types.

To determine the
common promotion type of the two types, use promote_type(Int8, UInt16) to find
whether it returns UInt16.

In [73]:
promote_type(Int8, UInt16)

UInt16

This is because, somewhere in the standard library, the following promote_rule function
was defined as promote_rule(::Type{Int8}, ::Type{Uint16}) = UInt16.

# The type hierarchy – subtypes and supertypes

Julia has a lot of built-in types, in fact, a whole hierarchy starting from the type
Any at the top. Every type in this structure also has a type, namely, DataType, so it is very
consistent. typeof(Any), typeof(Int64), typeof(Complex{Int64}), and
typeof(DataType) all return DataType. So, types in Julia are also objects; all concrete
types, except tuple types, which are a tuple of the types of its arguments, are of type
DataType.

In [75]:
typeof((String,Int64))

Tuple{DataType, DataType}

In [77]:
typeof(Tuple([String,Int64]))

Tuple{DataType, DataType}

This type hierarchy is like a tree; each type has one parent given by the supertype
function:

- supertype(Int64) returns Signed
- supertype(Signed) returns Integer
- supertype(Integer) returns Real
- supertype(Real) returns Numberm
- supertype(Number) returns Any
- supertype(Any) returns Any

In [83]:
supertype(Real)

Number

A type can have a lot of children or subtypes (a function from the InteractiveUtils
package) as follows:

In [84]:
using InteractiveUtils

In [86]:
subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

To indicate the subtype relationship, the operator < is used: Bool <: Integer and Bool
<: Any returns true, while Bool <: Char is false

In [87]:
Integer <: Number

true

# Concrete and abstract types

In this hierarchy, some types, such as Number, Integer, and Signed, are abstract, which
means that they have no concrete objects or values of their own.

Instead, objects or values
are of concrete types given by the result of applying typeof(value), such as Int8,
Float64, and String.

the concrete type of the value 5 is Int64 given by
typeof(5) (on a 64-bit machine).

In [90]:
typeof(10)

Int64

However, a value also has the type of all of its
supertypes; for example, isa(5, Number) returns true (

In [92]:
isa(5, Number)

true

Concrete types have no subtypes and might only have abstract types as their supertypes.
Schematically, we can differentiate them as follows:

An abstract type (such as Number and Real) is only a name that groups multiple subtypes
together, but it can be used as a type annotation or used as a type in array literals

In [93]:
a = Array{Number,}(undef,10)

10-element Vector{Number}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

Also,
note that an abstract type cannot have any fields.

The abstract type Any is the supertype of all types, and all the objects are also instances of
Any.

At the other end is Union{}: this type has no values and no subtypes. All types are
supertypes of Union{} , and no object is an instance of Union{}. It is unlikely that you will
ever have to use this type. The Nothing type has one value called nothing.

In [98]:
supertypes(Union)

(Union, Type{T}, Any)

In [99]:
subtypes(Union)

Type[]

In [100]:
subtypes(Nothing)

Type[]

In [101]:
supertypes(Nothing)

(Nothing, Any)

When a function is only used for its side-effects, convention dictates that it returns
nothing. We have seen this with the println function, where the printing is the side-
effect, for instance:

In [102]:
ret_val = print("WTF")

WTF

In [104]:
typeof(ret_val)

Nothing

# User-defined and composite types

In Julia, as a developer you can define your own types to structure data used in
applications. For example, if you need to represent points in a three-dimensional space, you
can define a type Point, as follows:

In [105]:
mutable struct Point 
    x::Float64
    y::Float64
    z::Float64
end


mutable here means that Point values can be modified. If your type values cannot be
changed, simply use struct.

In [107]:
a = Point(1,2,3)

Point(1.0, 2.0, 3.0)

In [111]:
subtypes(Point)

Type[]

In [112]:
typeof(a)

Point

Such a user-defined type is composed of a set of named fields with an optional type
annotation; that's why it is a composite type, and its type is also DataType.

In [113]:
a.x

1.0

If the type of a
named field is not given, then it is Any

A composite type is similar to struct in C, or a
class without methods in Java.

???????????????Unlike in other object-oriented languages such as Python or Java, where you call a function
on an object such as object.func(args), Julia uses the func(object, args) syntax.????????????????????????


In [115]:
a.y

2.0

y is not a method called on this data type :|

Julia has no classes (as types with functions belong to that type); this keeps the data and
functions separate. Functions and methods for a type will be defined outside that type.
Methods cannot be tied to a single type, because multiple dispatch connects them with
different types. This provides more flexibility, because when adding a new method for a
type, you don't have to change the code of the type itself, as you would have to do with the
code of the class in object-oriented languages.

The names of the fields that belong to a composite type can be obtained with the names
function of the type or object: fieldnames(Point) or fieldnames(typeof(p1)) returns
(:x, :y, :z).

In [117]:
fieldnames(Point)

(:x, :y, :z)

In [119]:
fieldnames(typeof(a))

(:x, :y, :z)

A user-defined type has two default implicit constructors that have the same name as the
type and take an argument for each field. You can see this by asking for the methods of
Point: methods(Point);

In [121]:
methods(Point)

# 2 methods for type constructor:
[1] Point(x::Float64, y::Float64, z::Float64) in Main at In[105]:2
[2] Point(x, y, z) in Main at In[105]:2

In [125]:
function distance(point::Point,)
    print(point.x)
end



distance (generic function with 1 method)

In [126]:
methods(Point)

# 2 methods for type constructor:
[1] Point(x::Float64, y::Float64, z::Float64) in Main at In[105]:2
[2] Point(x, y, z) in Main at In[105]:2

Fields that together contain the state of the object can be accessed by name, as in: p1.y,
which returns 4.0.

In [127]:
a.x

1.0

Objects of such a type are mutable, for example, I can change the z field to a new value with
p1.z = 3.14, resulting in p1 now having the value Point(2.0, 4.0, 3.14). Of course,
types are checked: p1.z = "A" results in an error.

Objects as arguments to functions are passed by reference, so that they can be changed inside
the function

If you don't want your objects to be changeable, replace type with the keyword struct,
for example:

In [128]:
struct ImmutablePoint
    x::Float64
    y::Float64
    z::Float64
end


In [129]:
b = ImmutablePoint(1,2,3)

ImmutablePoint(1.0, 2.0, 3.0)

In [130]:
b.x = 7

LoadError: setfield!: immutable struct of type ImmutablePoint cannot be changed

Immutable types enhance performance, because Julia can optimize the
code for them. Another big advantage of immutable types is thread safety:
an immutable object can be shared between threads without needing
synchronization. If, however, such an immutable type contains a mutable
field such as an array, the contents of that field can be changed. So, define
your immutable types without mutable fields.

A type once defined cannot be changed. If we try to define a new type Point with fields of
type Int64, or with added fields, we get an ERROR: invalid redefinition of
constant TypeName error message.

In [131]:
mutable struct Point
    x::Float64
    y::Float64
    z::Float64
    a::Float64
end

LoadError: invalid redefinition of constant Point

A new type that is exactly the same as an existing type can be defined as an alias through a
simple assignment, for instance, Point3D = Point. Now, objects of type Point3D can also
be created: p31 = Point3D(1, 2, 3) returns Point(1.0, 2.0, 3.0). Julia also uses
this internally; the alias Int is used for either Int64 or Int32, depending on the
architecture of the system that is being used.

# When are two values or objects equal or identical?

Whether two values are equal or not can be decided by the == operator, for example, 5 ==
5 and 5 == 5.0 are both true. Equivalent to this operator is the isequal() function:

In [1]:
5 == 5.0

true

In [2]:
isequal(4,4)

true

In [3]:
isequal(4.0,4)

true

Both the preceding statements return true, because objects such as numbers are immutable
and they are compared at the bits level.

To see whether the two objects x and y are identical, they must be compared with the ===
operator.

In [4]:
5 === 5

true

In [5]:
5 === 5.0

false

For objects that are more complex, such as strings, arrays, or objects that are constructed
from composite types, the addresses in the memory are compared to check whether they
point to the same memory location.

For immutable object such as struct, this gets
optimized so that instances with the same value point to the same object:

In [6]:
struct Vector3D
    x::Float64
    y::Float64
    z::Float64
end


In [7]:
a = Vector3D(1,2,3)

Vector3D(1.0, 2.0, 3.0)

In [8]:
b = Vector3D(1,2,3)

Vector3D(1.0, 2.0, 3.0)

In [9]:
a == b

true

In [10]:
a === b

true

In [11]:
b = Vector3D(1, 3, 3)

Vector3D(1.0, 3.0, 3.0)

In [12]:
a == b

false

In [13]:
a === b

false

However, if objects are mutable, they are different objects even if they have the same value,
as follows:

In [14]:
mutable struct MVector3D
    x::Float64
    y::Float64
    z::Float64
end


In [15]:
m = MVector3D(1,2,3)
n = MVector3D(1,2,3)

MVector3D(1.0, 2.0, 3.0)

In [16]:
isequal(m, n)

false

In [17]:
m == n

false

In [18]:
m === n

false

# A multiple-dispatch example

Let's now explore an example about people working in a company to show multiple
dispatch in action. Let's define an abstract type Employee and a type Developer that is a
subtype:

In [24]:
abstract type Employee 
end

We cannot make objects from an abstract type: calling Employee() only returns an ERROR:
MethodError: no constructors have been defined for Employee error message.

In [34]:
Employee()

LoadError: MethodError: no constructors have been defined for Employee

In [27]:
mutable struct Developer <: Employee
    name::String 
    iq::UInt8
    favorite_language::String
end


The type Developer has two implicit constructors, but we can define another outer
constructor, which uses a default constructor as follows:

In [35]:
Developer(name, iq) = Developer(name, iq, "Java")

Developer

In [36]:
methods(Developer)

# 3 methods for type constructor:
[1] Developer(name::String, iq::UInt8, favorite_language::String) in Main at In[27]:2
[2] Developer(name, iq) in Main at In[35]:1
[3] Developer(name, iq, favorite_language) in Main at In[27]:2

In [29]:
a = Developer("javid",110,"WTF")

Developer("javid", 0x6e, "WTF")

In [30]:
typeof(a)

Developer

In [31]:
supertypes(typeof(a))

(Developer, Employee, Any)

In [33]:
isa(a,Employee)

true

In [42]:
dev11 = Developer("Anna",150)
dev12 = Developer("javid", 120)

Developer("javid", 0x78, "Java")

Similarly, we can define a type Manager and an instance of it as follows:

In [43]:
mutable struct Manager
    name::String
    iq::UInt8
    favorite_language::String
end



In [44]:
man1 = Manager("Krados",160,"haskell")

Manager("Krados", 0xa0, "haskell")

Concrete types, such as Developer or Manager, cannot be subtyped:

In [45]:
struct MobileDeveloper <: Developer
    platform::String
end


LoadError: invalid subtyping in definition of MobileDeveloper

In [46]:
cleverness(emp::Employee) = Int32(emp.iq)

cleverness (generic function with 1 method)

In [47]:
cleverness(dev11)

150

In [49]:
methods(Developer)

# 3 methods for type constructor:
[1] Developer(name::String, iq::UInt8, favorite_language::String) in Main at In[27]:2
[2] Developer(name, iq) in Main at In[35]:1
[3] Developer(name, iq, favorite_language) in Main at In[27]:2

In [50]:
cleverness(man1)

LoadError: MethodError: no method matching cleverness(::Manager)
[0mClosest candidates are:
[0m  cleverness([91m::Employee[39m) at In[46]:1

If we now define a function cleverness as cleverness(emp::Employee) = emp.iq,
then cleverness(devel1) returns 110, but cleverness(man1) returns an ERROR:
MethodError: `cleverness` has no method matching cleverness(::Manager)
error message; the function has no method for a manager.

The following function makes us think that managers are always cleverer, which is, of
course, not true:

In [53]:
function cleverer(m::Manager, e::Employee)
    print("The Manager $(m.name) is cleverer! in my ass")
end


cleverer (generic function with 1 method)

In [54]:
cleverer(man1,dev11)

The Manager Krados is cleverer! in my ass

In [57]:
function cleverer(d::Developer, e::Employee)
    print("The Developer $(d.name) is smarter :)")
end


cleverer (generic function with 2 methods)

In [58]:
cleverer(dev11,dev12)

The Developer Anna is smarter :)

It matches a method
because devel2 is also an employee.

In [60]:
cleverer(dev11,man1)

LoadError: MethodError: no method matching cleverer(::Developer, ::Manager)
[0mClosest candidates are:
[0m  cleverer(::Developer, [91m::Employee[39m) at In[57]:1

In [62]:
function cleverer(e::Employee, d::Developer)
    if e.iq <= d.iq 
        print("$(e.name) is smarter than $(b.name)")
    else
        print("$(d.name) is smarter than $(e.name)")
    end
end


cleverer (generic function with 3 methods)

In [63]:
cleverer(dev11,dev12)

LoadError: MethodError: cleverer(::Developer, ::Developer) is ambiguous. Candidates:
  cleverer(d::Developer, e::Employee) in Main at In[57]:1
  cleverer(e::Employee, d::Developer) in Main at In[62]:1
Possible fix, define
  cleverer(::Developer, ::Developer)

Devs are both Developer and Evmployee so the compiler doesn't know which to choose

In [66]:
function cleverer(d1::Developer, d2::Developer)
    if d1.iq <= d2.iq 
        print("$(d2.name) is smarter than $(d1.name)")
    else
        print("$(d1.name) is smarter than $(d2.name)")
    end
end


cleverer (generic function with 4 methods)

In [67]:
cleverer(dev11,dev12)

Anna is smarter than javid

Always avoid method ambiguities by specifying an appropriate method
for the intersection case.

# Types and collections – inner constructors

In [1]:
mutable struct Person 
    firstname::String
    lastname::String
    sex::String
    age::Int
    Children::Array{String,1}
end

    

In [2]:
p1 = Person("javid", "norouzi", "male", 28, ["Javid the second", "Javid the third"])

Person("javid", "norouzi", "male", 28, ["Javid the second", "Javid the third"])

This example demonstrates that an object can contain collections, such as arrays or
dictionaries. Custom types can also be stored in a collection, just like built-in types, for
example:

In [3]:
peoples = Person[]

Person[]

In [4]:
push!(peoples, Person("a","b","female",100,["NoBody"]))

1-element Vector{Person}:
 Person("a", "b", "female", 100, ["NoBody"])

In [5]:
fullname(p::Person) = "$(p.firstname) $(p.lastname)"

fullname (generic function with 1 method)

In [6]:
fullname(p1)

"javid norouzi"

Or, slightly more performant:

In [7]:
fullname(p::Person) = string(p.firstname, " ", p.lastname)

fullname (generic function with 1 method)

In [8]:
fullname(p1)

"javid norouzi"

If you need to include error checking or transformations as part of the type construction
process, you can use inner constructors (so-called because they are defined inside the type
itself)

In [10]:
mutable struct Family
    name::String
    members::Array{String,1}
    big::Bool
    Family(name::String) = new(name, String[], false)
    Family(name::String, members) = new(name, members, length(members)>4)
end


In [11]:
fam = Family("Bates-Smith", ["Alan", "Julia", "Jeff", "Stephan","Viral"])

Family("Bates-Smith", ["Alan", "Julia", "Jeff", "Stephan", "Viral"], true)

The keyword new can only be used in an inner constructor to create an object of the
enclosing type.

The first constructor takes one argument and generates a default for the
other two values. The second constructor takes two arguments and infers the value of big.`

Inner constructors give you more control over how the values of the type can be created.
Here, they are written with the short function notation, but if they are multiline, they will
use the normal function syntax.

Note that when you use inner constructors, there are no default constructors any more.
Outer constructors, calling a limited set of inner constructors, are often the best practice.

# Type unions

In geometry, a two-dimensional point and a vector are not the same, even if they both have
an x and y component. In Julia, we can also define them as different types, as follows:

In [12]:
mutable struct Point
    x::Float64
    y::Float64
end


In [13]:
mutable struct Vector2D
    x::Float64
    y::Float64
end

In [14]:
p = Point(2,3)
v = Vector2D(3,4)

Vector2D(3.0, 4.0)

In [15]:
+(p, v)

LoadError: MethodError: no method matching +(::Point, ::Vector2D)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /opt/julia-1.7.1/share/julia/base/operators.jl:655

To define a + method here, first do an import Base.+

In [17]:
import Base.+


In [19]:
function +(p::Point,v::Vector2D)
    return Point(p.x + v.x ,p.y + v.y)
end


+ (generic function with 209 methods)

In [20]:
+(p,v)

Point(5.0, 7.0)

Even after defining the following, +(p, v) still returns the same error because of multiple
dispatch. Julia has no way of knowing that +(p,v) should be the same as +(v,p):

In [21]:
+(p::Point, q::Point) = Point(p.x + q.x, p.y + q.y)
+(u::Vector2D, v::Vector2D) = Point(u.x + v.x, u.y + v.y)
+(u::Vector2D, p::Point) = Point(u.x + p.x, u.y + p.y)

+ (generic function with 212 methods)

Now you can ask the question: Don't multiple dispatch and many types give rise to code
duplication, as is the case here?

The answer is no, because, in such a case, we can define a union type, VecOrPoint:

In [23]:
VecOrPoint = Union{Vector2D, Point}


Union{Point, Vector2D}

In [24]:
+(a::VecOrPoint, b::VecOrPoint) = VecOrPoint(a.x + b.x, a.y + b.y)

+ (generic function with 213 methods)

In [25]:
p + v

Point(5.0, 7.0)

In [26]:
v + p

Point(5.0, 7.0)

In [27]:
+(v,p)

Point(5.0, 7.0)

In [28]:
+(p,v)

Point(5.0, 7.0)

# Parametric types and methods

An array can take elements of different types. Therefore, we can have, for example, arrays
of the following types: Array{Int64,1}, Array{Int8,1}, Array{Float64,1}, or
Array{String, 1}, and so on. That is why an Array is a parametric type; its elements can
be of any arbitrary type T, written as Array{T, 1}.

In general, types can take type parameters, so that type declarations actually introduce a
whole family of new types. Returning to the Point example of the previous section, we can
generalize it to the following:

In [30]:
mutable struct PPoint{T}
    x::T
    y::T
end

In [31]:
p = PPoint{Int64}(1,2)

PPoint{Int64}(1, 2)

This abstract type creates a whole family of new possible concrete types (but they are only
compiled as needed at runtime), such as Point{Int64}, Point{Float64}, and
Point{String}.

These are all subtypes of Point: Point{String} <: Point returns true. However, this
is not the case when comparing different Point types, whose parameter types are subtypes
of one another: Point{Float64} <: Point{Real} returns false.

In [32]:
PPoint{String} <: PPoint

true

In [33]:
Point <: PPoint

false

In [34]:
PPoint{Float64} <: PPoint{Number}

false

To construct objects, you can indicate the type T in the constructor, as in p =
Point{Int64}(2, 5), but this can be shortened to p = Point(2, 5). Or let's consider
another example: p = Point("London", "Great-Britain").

If you want to restrict the parameter type T to only the subtypes of Real, this can be written
as follows:

In [36]:
mutable struct PPPoint{T <: Real }
    x::T
    y::T
end

In [39]:
p = PPoint("W", "j")

PPoint{String}("W", "j")

In [41]:
P = PPPoint(2, 3)

PPPoint{Int64}(2, 3)

In much the same way, methods can also optionally have type parameters immediately
after their name and before the tuple of arguments. For example, to constrain two
arguments to be of the same type T, run the following command:

In [45]:
add(x::T, y::T) where T = x + y

add (generic function with 1 method)

the = is for the functino 

In [47]:
function add(x::T, y::T) where T
    x + y
end


add (generic function with 1 method)

In [48]:
add(2,3)

5

In [49]:
add(2,3.)

LoadError: MethodError: no method matching add(::Int64, ::Float64)
[0mClosest candidates are:
[0m  add(::T, [91m::T[39m) where T at In[47]:1

In [1]:
function add(x::T, y::T) where T <: Number  
    x + y
end


add (generic function with 1 method)

In [2]:
isa(3.0, Number)

true

In [3]:
isa(2, Number)

true

In [20]:
add(x::T , y::T)   where T <: Number =    x + y



add (generic function with 1 method)

In [24]:
add(2.,3)

LoadError: MethodError: no method matching add(::Float64, ::Int64)
[0mClosest candidates are:
[0m  add(::T, [91m::T[39m) where T<:Number at In[20]:1

In [22]:
methods(add)

# 1 method for generic function "add":
[1] add(x::T, y::T) where T<:Number in Main at In[20]:1

In [11]:
function vecfloat(x::Vector{T}) where T <: AbstractFloat
# code
end

vecfloat (generic function with 1 method)

In [12]:
a = Vector{Int64}([1,2,3,4])

4-element Vector{Int64}:
 1
 2
 3
 4

In [13]:
vecfloat(a)

LoadError: MethodError: no method matching vecfloat(::Vector{Int64})
[0mClosest candidates are:
[0m  vecfloat([91m::Vector{T}[39m) where T<:AbstractFloat at In[11]:1

In [14]:
a = Vector{Float64}([1,2,3,4])

4-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0

In [25]:
vecfloat(a)

# Standard modules and paths

The code for Julia packages (also called libraries) is contained in a module whose name
starts with an uppercase letter by convention, like this:

This serves to separate all its definitions from those in other modules so that no name
conflicts occur. Name conflicts are solved by qualifying the function by the module name.
For example, the packages Winston and Gadfly both contain a function plot. If we needed
these two versions in the same script, we would write it as follows:m

In [28]:
# import Winston
# import Gadfly
# Winston.plot(rand(4))
# Gadfly.plot(x=[1:10], y=rand(10))

All variables defined in the global scope are automatically added to the Main module.
Thus, when you write x = 2 in the REPL, you are adding the variable x to the Main
module.

Julia starts with Main as the current top-level module. The module Core is a set of non-Julia
sources (in the src directory of the GitHub source); for example, C/C++ and Femtolisp,
which are used to create libjulia, are used by the Julia source to interface to the OS
through the API. The standard library is also available. The code for the standard library
(the contents of /base) is contained in the following modules:

The type of a module is Module: typeof(Base), which returns Module.

In [29]:
typeof(Base)

Module

If we call
names(Main), we get, for example, 5-element Array{Symbol,1}: :ans, :Main,
:Core, :Base, and :InteractiveUtils. If you have defined other variables or functions
in the REPL, these would also show up.

In [31]:
names(Main)

6-element Vector{Symbol}:
 :Base
 :Core
 :Main
 :a
 :add
 :vecfloat

All the top-level defined variables and functions, together with the default modules, are
stored as symbols.

In [32]:
varinfo()

| name     |     size | summary                                   |
|:-------- | --------:|:----------------------------------------- |
| Base     |          | Module                                    |
| Core     |          | Module                                    |
| Main     |          | Module                                    |
| a        | 72 bytes | 4-element Vector{Float64}                 |
| add      |  0 bytes | add (generic function with 1 method)      |
| vecfloat |  0 bytes | vecfloat (generic function with 1 method) |


This can also be used for another module. For example, varinfo(Winston) lists all the
exported names from the module Winston.

In [33]:
varinfo(Winston)

LoadError: UndefVarError: Winston not defined

A module can make some of its internal definitions (such as constants, variables, types,
functions, and so on) visible to other modules (as if making them public) by declaring them
with export. This can be seen in the following example:

In [34]:
# export Type1, perc

As we saw in Chapter 1, Installing the Julia Platform, a module can also include other source
files in their entirety with include("file1.jl"). However, this means that the included
files are not modules. Using include("file1.jl") is, to the compiler, no different from
copying file1.jl and pasting it directly in the current file or the REPL.

In general, use import to import definitions from another module in the current module:

- After import.LibA, you can use all definitions from LibA inside the current
module by qualifying them with LibA., such as LibA.a

- The import LibB.varB or import LibD.funcD statement only imports one
name; the function funcD must be used as LibD.funcD.

- Use importall LibE to import all the exported names from LibE in the current
module.

In [35]:
;subl TemperatureConverter.jl

In [36]:
include("TemperatureConverter.jl")

Main.TemperatureConverter

In [37]:
println("$(TemperatureConverter.as_celsius(100, :Celsius))")

100


Imported variables are read-only, and the current module cannot create variables with the
same names as the imported ones

A source file can contain many modules, or one module
can be defined in several source files. If a module contains a function __init__(), this will
be executed when the module is first loaded.

The variable LOAD_PATH contains a list of directories where Julia looks for (module) files
when running the using, import, or include statements.

Put this statement in a file to extend LOAD_PATH on every Julia startup:
push!(LOAD_PATH, "new/path/to/search")